# Experimental

Experiments before they're integrated into the main codebase.

In [2]:
import os
import requests

In [8]:
PWD = os.getcwd()
DATA_DIR = os.path.join(PWD, "data")
INPUT_DATA_URL = ("https://raw.githubusercontent.com/karpathy/char-rnn/"
                  "master/data/tinyshakespeare/input.txt")

In [9]:
def fetch_input_data() -> str:
    """Gets the input data, caching it for easy access."""
    input_file_path = os.path.join(DATA_DIR, "input.txt")
    if not os.path.exists(input_file_path):
        with open(input_file_path, "w", encoding="utf-8") as f:
            f.write(requests.get(INPUT_DATA_URL).text)
    
    with open(input_file_path, "r", encoding="utf-8") as f:
        return f.read()

In [10]:
input_data = fetch_input_data()

In [11]:
print(len(input_data))

1115394
